In [2]:
%pip install accelerate peft bitsandbytes transformers trl datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 88.5 MB/s eta 0:00:00
     ━━━━━━━

In [3]:
# load libs
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer
import warnings
warnings.simplefilter("ignore")

In [4]:
# define global vars

llm_model_name = "meta-llama/Llama-2-7b-chat-hf"
dataset_name = "ZahrizhalAli/mental_health_conversational_dataset"
finetuned_model = "llama-2-7b-chat-mental_health"
token = "<TOKEN>"
proj_fold = os.getcwd()
cache_dir = os.path.join(proj_fold,"llm_models")
data_cache_dir = os.path.join(proj_fold,"llm_data")
results_dir = os.path.join(proj_fold,"results")

In [5]:
# load the quant model
quant_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(llm_model_name, quantization_config=quant_config, cache_dir=cache_dir,token=token)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
# load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(llm_model_name, cache_dir=cache_dir,token=token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
# load the data
dataset = load_dataset(dataset_name,cache_dir=data_cache_dir)

Generating train split:   0%|          | 0/175 [00:00<?, ? examples/s]

In [8]:
# peft params define
peft_params = LoraConfig(lora_alpha=16,lora_dropout=0.1,r=64,bias="none",task_type="CAUSAL_LM")

In [14]:
# define train params

training_params = TrainingArguments(
    output_dir=results_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [10]:
def preprocess_text(sample):

    text = sample["text"]
    text = text.replace("<HUMAN>:","<s>[INST]").replace("\n<ASSISTANT>:","[/INST]")+"</s>"
    return {"processed_text": text}

In [11]:
preprocessed_dataset = dataset.map(preprocess_text)

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

In [15]:
# define trainer

trainer = SFTTrainer(
    model=model,
    train_dataset=preprocessed_dataset["train"],
    peft_config=peft_params,
    dataset_text_field="processed_text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

Map:   0%|          | 0/175 [00:00<?, ? examples/s]

In [17]:
torch.cuda.empty_cache()

In [18]:
trainer.train()

Step,Training Loss
25,1.277600
50,1.463100
75,1.336100
100,1.119600
125,1.298800
150,1.035900
175,1.419700


TrainOutput(global_step=175, training_loss=1.2786833626883372, metrics={'train_runtime': 517.3642, 'train_samples_per_second': 0.338, 'train_steps_per_second': 0.338, 'total_flos': 1750407907491840.0, 'train_loss': 1.2786833626883372, 'epoch': 1.0})

In [23]:
preprocessed_dataset["train"][-2]["processed_text"]

"<s>[INST] What to do if i'm worried about my mental health?/n<ASSISTANT>: The most important thing is to talk to someone you trust. This might be a friend, colleague, family member, or GP. In addition to talking to someone, it may be useful to find out more information about what you are experiencing. These things may help to get some perspective on what you are experiencing, and be the start of getting help.</s>"

In [24]:
prompt = "I am suffering from anxiety issues. What can I do?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] I am suffering from anxiety issues. What can I do? [/INST] I'm so sorry to hear that you're struggling with anxiety. It's completely understandable to feel overwhelmed and unsure of where to turn.

Firstly, it's important to remember that you're not alone in this. Many people struggle with anxiety, and there are resources available to help you cope with these feelings.

Here are some suggestions that might be helpful:

1. Seek Professional Help: Consider talking to a therapist or counselor who specializes in anxiety. They can provide you with personalized guidance and support.

2. Practice Relaxation Techniques: There are various relaxation techniques you can try, such as deep breathing, progressive muscle relaxation, or guided imagery. Find what works best for you and practice regularly.



In [26]:
trainer.model.save_pretrained(finetuned_model)
trainer.tokenizer.save_pretrained(finetuned_model)

('llama-2-7b-chat-mental_health/tokenizer_config.json',
 'llama-2-7b-chat-mental_health/special_tokens_map.json',
 'llama-2-7b-chat-mental_health/tokenizer.model',
 'llama-2-7b-chat-mental_health/added_tokens.json',
 'llama-2-7b-chat-mental_health/tokenizer.json')

In [ ]:
# import shutil
# from google.colab import files

# folder_path = "/content/llama-2-7b-chat-mental_health"
# shutil.make_archive("/content/llama-2-7b-chat-mental_health", 'zip', folder_path)
# files.download("/content/llama-2-7b-chat-mental_health.zip")